In [1]:
import os
import logging
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.tensorflow.serving import Model

In [2]:
sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [39]:
estimator = TensorFlow(entry_point='model.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.m5.xlarge',
                             #framework_version='2.2.0',
                             py_version='py3',
                             source_dir='./train',
                             image_name='763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.2.0-gpu-py37-cu102-ubuntu18.04',
                             distributions={'parameter_server': {'enabled': True}})

No framework_version specified, defaulting to version 1.11. This is not the latest supported version. If you would like to use version 2.1.0, please add framework_version=2.1.0 to your constructor.


In [40]:
estimator.fit()

2020-07-04 19:57:16 Starting - Starting the training job...
2020-07-04 19:57:19 Starting - Launching requested ML instances.........
2020-07-04 19:58:52 Starting - Preparing the instances for training...
2020-07-04 19:59:30 Downloading - Downloading input data
2020-07-04 19:59:30 Training - Downloading the training image.........
2020-07-04 20:01:09 Training - Training image download completed. Training in progress..2020-07-04 20:01:12,210 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2020-07-04 20:01:12,216 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-07-04 20:03:17,950 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/usr/local/bin/python3.7 -m pip install -r requirements.txt
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3754 sha256=e961cfedadf3a85ee00304162badb0b0ef6c479a8f349b00d97d281f4b8f8e11
  Stored in directory: /root/.cache/pip/wh

In [ ]:
#predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')

In [47]:
role

'arn:aws:iam::378706508412:role/service-role/AmazonSageMaker-ExecutionRole-20200607T125385'

In [100]:
model_location = 's3://sagemaker-us-west-2-378706508412/tensorflow-training-2020-07-04-19-49-13-246/output/model.tar.gz'

deployment_model = Model(
              entry_point='inference.py',
              model_data=model_location,
              #framework_version='2.2.0'
              source_dir='./serve',
              container_log_level = logging.DEBUG,
              image='763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-inference:2.2.0-gpu-py37-cu102-ubuntu18.04',
              #image='763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-inference:1.15.2-gpu-py36-cu100-ubuntu18.04',
              role=role)

predictor = deployment_model.deploy(initial_instance_count=1, instance_type='ml.t2.2xlarge')


-----------------!

In [106]:
predictor.predict({'claim_text': 'masks spread the coronavirus'})

{'predictions': [[0.000627173402]]}

In [104]:
predictor.endpoint

'tensorflow-inference-2020-07-07-05-29-55-973'